# What this file does

Looks at facility level data provided by Health and Human Services to assess hospital capacity. Skip to the "Formulae" section to see results.

In [1]:
import pandas as pd

# Tools

In [2]:
df = pd.read_csv('reported_hospital_capacity_admissions_facility-level_weekly_average_timeseries_20201207 (1).csv')

In [3]:
#columns;pediatric estimates seem to be negative (suppressed?)

In [21]:
df.columns

Index(['hospital_pk', 'collection_week', 'state', 'ccn', 'hospital_name',
       'address', 'city', 'zip', 'hospital_subtype', 'fips_code',
       'is_metro_micro', 'total_beds_7_day_avg',
       'all_adult_hospital_beds_7_day_avg',
       'all_adult_hospital_inpatient_beds_7_day_avg',
       'inpatient_beds_used_7_day_avg',
       'all_adult_hospital_inpatient_bed_occupied_7_day_avg',
       'total_adult_patients_hospitalized_confirmed_and_suspected_covid_7_day_avg',
       'total_adult_patients_hospitalized_confirmed_covid_7_day_avg',
       'total_pediatric_patients_hospitalized_confirmed_and_suspected_covid_7_day_avg',
       'total_pediatric_patients_hospitalized_confirmed_covid_7_day_avg',
       'inpatient_beds_7_day_avg', 'total_icu_beds_7_day_avg',
       'total_staffed_adult_icu_beds_7_day_avg', 'icu_beds_used_7_day_avg',
       'staffed_adult_icu_bed_occupancy_7_day_avg',
       'staffed_icu_adult_patients_confirmed_and_suspected_covid_7_day_avg',
       'staffed_icu_adult_p

# Filtering

In [4]:
df_ct= df[df['state']=='CT']

In [5]:
df['collection_week'] =pd.to_datetime(df['collection_week'])

This date marks the start of the collection week

In [6]:
df_ct['collection_week'].max()

'2020-11-27'

In [7]:
df_ct[df_ct['collection_week'] == df_ct['collection_week'].max()]['city'].unique()

array(['DANBURY', 'NORWALK', 'BRIDGEPORT', 'GREENWICH', 'STAMFORD',
       'NEW BRITAIN', 'HARTFORD', 'MANCHESTER', 'BRISTOL', 'FARMINGTON',
       'TORRINGTON', 'SHARON', 'MIDDLETOWN', 'NEW HAVEN', 'BRANFORD',
       'WATERBURY', 'MERIDEN', 'DERBY', 'WALLINGFORD', 'NEW LONDON',
       'NORWICH', 'STAFFORD SPRINGS', 'ROCKVILLE', 'PUTNAM',
       'WILLIMANTIC'], dtype=object)

In [8]:
df_latest = df_ct[df_ct['collection_week'] == df_ct['collection_week'].max()]

In [9]:
#comparing with CHA list:
 # HOSPITAL FOR SPECIAL CARE, GAYLORD HOSPITAL INC and CONNECTICUT HOSPICE INC,THE don't feature in their list of acute hospitals

In [10]:
df_latest['hospital_name'].unique()

array(['DANBURY HOSPITAL', 'NORWALK HOSPITAL',
       "ST VINCENT'S MEDICAL CENTER", 'GREENWICH HOSPITAL ASSOCIATION -',
       'BRIDGEPORT HOSPITAL', 'STAMFORD HOSPITAL',
       'HOSPITAL FOR SPECIAL CARE', 'HARTFORD HOSPITAL',
       'CONNECTICUT CHILDRENS MEDICAL CENTER',
       'MANCHESTER MEMORIAL HOSPITAL', 'BRISTOL HOSPITAL',
       'HOSPITAL OF CENTRAL CONNECTICUT, THE', 'JOHN DEMPSEY HOSPITAL',
       'ST FRANCIS HOSPITAL & MEDICAL CENTER',
       'CHARLOTTE HUNGERFORD HOSPITAL', 'SHARON HOSPITAL',
       'MIDDLESEX HOSPITAL', 'YALE-NEW HAVEN HOSPITAL',
       'CONNECTICUT HOSPICE INC,THE', "SAINT MARY'S HOSPITAL",
       'MIDSTATE MEDICAL CENTER', 'GRIFFIN HOSPITAL',
       'GAYLORD HOSPITAL INC', 'WATERBURY HOSPITAL',
       'LAWRENCE & MEMORIAL HOSPITAL', 'WILLIAM W BACKUS HOSPITAL',
       'JOHNSON MEMORIAL HOSPITAL', 'ROCKVILLE GENERAL HOSPITAL',
       'DAY KIMBALL HOSPITAL', 'WINDHAM COMM MEM HOSP & HATCH HOSP'],
      dtype=object)

In [11]:
df_latest['city'].value_counts()

HARTFORD            3
BRIDGEPORT          2
NEW BRITAIN         2
WATERBURY           2
STAMFORD            1
BRANFORD            1
NORWICH             1
MERIDEN             1
DANBURY             1
NEW LONDON          1
WILLIMANTIC         1
FARMINGTON          1
WALLINGFORD         1
NEW HAVEN           1
NORWALK             1
PUTNAM              1
ROCKVILLE           1
MANCHESTER          1
GREENWICH           1
BRISTOL             1
SHARON              1
TORRINGTON          1
MIDDLETOWN          1
DERBY               1
STAFFORD SPRINGS    1
Name: city, dtype: int64

# Formulae to calculate hospital capacity

GitHub documentation of dataset; negative values indicate suppressed values
https://github.com/CareSet/COVID_Hospital_PUF?src=hd

Where HHS results may differ from CHA values:
- using 7 day averages for all numbers, not daily (dataset updated once a week) 
- different ways of handling missing values; numbers reported to HHSdaily and aggregated over the week; discrepancies may arise if a hospital misses a day of reporting, say.
- using staffed, adult beds in ICU nummbers
- inpatient beds do not make a distinction between adult and staffed in the same way 
- definition of a bed 
- three facilities in this dataset do not appear in CHA's list

In [12]:
def calculate_hosp_icu_capacity(hosp):
    return pd.Series(df_latest[df_latest['hospital_name'] == hosp]['staffed_adult_icu_bed_occupancy_7_day_avg']/df_latest[df_latest['hospital_name'] == hosp]['total_staffed_adult_icu_beds_7_day_avg']).values[0]

In [13]:
def calculate_hosp_icus(hosp):
    return pd.Series(df_latest[df_latest['hospital_name'] == hosp]['total_staffed_adult_icu_beds_7_day_avg']).values[0]

In [14]:
def calculate_hosp_inpatient_capacity(hosp):
    return pd.Series(df_latest[df_latest['hospital_name'] == hosp]['inpatient_beds_used_7_day_avg']/df_latest[df_latest['hospital_name'] == hosp]['inpatient_beds_7_day_avg']).values[0]

In [15]:
def calculate_hosp_inpatient_beds(hosp):
    return pd.Series(df_latest[df_latest['hospital_name'] == hosp]['inpatient_beds_7_day_avg']).values[0]

In [16]:
def city(hosp):
    return pd.Series(df_latest[df_latest['hospital_name'] == hosp]['city']).values[0]

In [17]:
calculate_hosp_icu_capacity('STAMFORD HOSPITAL')

0.809375

In [18]:
calculate_hosp_inpatient_capacity('STAMFORD HOSPITAL')

0.8593489148580968

In [19]:
list_hosp_capacity = []
for hosp in list(df_latest['hospital_name'].unique()):
    dict_hosp = {}
    dict_hosp['name']= hosp
    dict_hosp['city']= city(hosp)
    dict_hosp['% Adult, Staffed ICUs Occupied'] = round(100*calculate_hosp_icu_capacity(hosp),2)
    dict_hosp['% Inpatient beds used'] = round(100*calculate_hosp_inpatient_capacity(hosp),2)
    dict_hosp['ICUs Available'] = calculate_hosp_icus(hosp)
    dict_hosp['Inpatient beds available'] = calculate_hosp_inpatient_beds(hosp)
    list_hosp_capacity += [dict_hosp]


In [20]:
#notes on data: negative values indicate suppressed numbers; percentages may exceed 100 if a hospital is operating above capacity
#

In [82]:
pd.DataFrame(list_hosp_capacity)

,name,city,"% Adult, Staffed ICUs Occupied",% Inpatient beds used,ICUs Available,Inpatient beds available
0,DANBURY HOSPITAL,DANBURY,80.50,80.51,20.0,319.6
1,NORWALK HOSPITAL,NORWALK,36.33,49.25,30.0,293.0
2,ST VINCENT'S MEDICAL CENTER,BRIDGEPORT,28.64,69.90,66.0,315.6
3,GREENWICH HOSPITAL ASSOCIATION -,GREENWICH,38.67,64.83,30.0,205.0
4,BRIDGEPORT HOSPITAL,BRIDGEPORT,86.30,50.46,54.0,735.0
5,STAMFORD HOSPITAL,STAMFORD,80.94,85.93,32.0,239.6
6,HOSPITAL FOR SPECIAL CARE,NEW BRITAIN,100.00,90.76,-999999.0,237.0
7,HARTFORD HOSPITAL,HARTFORD,46.58,87.39,158.0,846.3
8,CONNECTICUT CHILDRENS MEDICAL CENTER,HARTFORD,100.00,63.06,-999999.0,183.0
9,MANCHESTER MEMORIAL HOSPITAL,MANCHESTER,86.82,71.28,22.0,211.0


Questions:
- Find source of discrepancy with CHA data
- Examine calculations of state-level estimates (https://protect-public.hhs.gov/datasets/state-representative-estimates-for-hospital-utilization/data?geometry=89.569%2C-16.702%2C-96.408%2C72.161&orderBy=state_name)
- Aggregate on city level? Have to figure out how to handle suppressed values
- What's the story here? 
- Further analysis: Dataset includes age breakdowns of COVID patients, need to see how to clean it before I can work with it

# COVID in CT hospitals

Age breakdowns all suppressed

In [24]:
for column in list(df_latest.columns):
    if 'covid' in column:
        print(column)

total_adult_patients_hospitalized_confirmed_and_suspected_covid_7_day_avg
total_adult_patients_hospitalized_confirmed_covid_7_day_avg
total_pediatric_patients_hospitalized_confirmed_and_suspected_covid_7_day_avg
total_pediatric_patients_hospitalized_confirmed_covid_7_day_avg
staffed_icu_adult_patients_confirmed_and_suspected_covid_7_day_avg
staffed_icu_adult_patients_confirmed_covid_7_day_avg
total_patients_hospitalized_confirmed_influenza_and_covid_7_day_avg
total_adult_patients_hospitalized_confirmed_and_suspected_covid_7_day_sum
total_adult_patients_hospitalized_confirmed_covid_7_day_sum
total_pediatric_patients_hospitalized_confirmed_and_suspected_covid_7_day_sum
total_pediatric_patients_hospitalized_confirmed_covid_7_day_sum
staffed_icu_adult_patients_confirmed_and_suspected_covid_7_day_sum
staffed_icu_adult_patients_confirmed_covid_7_day_sum
total_patients_hospitalized_confirmed_influenza_and_covid_7_day_sum
total_adult_patients_hospitalized_confirmed_and_suspected_covid_7_day_co

In [43]:
df_covid_by_hosp = df_latest[['hospital_name','city','total_adult_patients_hospitalized_confirmed_and_suspected_covid_7_day_avg','total_adult_patients_hospitalized_confirmed_and_suspected_covid_7_day_sum']]

In [46]:
df_covid_by_hosp.reset_index()

,index,hospital_name,city,total_adult_patients_hospitalized_confirmed_and_suspected_covid_7_day_avg,total_adult_patients_hospitalized_confirmed_and_suspected_covid_7_day_sum
0,716,DANBURY HOSPITAL,DANBURY,74.7,523.0
1,717,NORWALK HOSPITAL,NORWALK,47.0,329.0
2,718,ST VINCENT'S MEDICAL CENTER,BRIDGEPORT,56.4,395.0
3,719,GREENWICH HOSPITAL ASSOCIATION -,GREENWICH,33.4,234.0
4,720,BRIDGEPORT HOSPITAL,BRIDGEPORT,138.7,971.0
5,721,STAMFORD HOSPITAL,STAMFORD,44.3,310.0
6,722,HOSPITAL FOR SPECIAL CARE,NEW BRITAIN,-999999.0,10.0
7,723,HARTFORD HOSPITAL,HARTFORD,121.6,851.0
8,724,CONNECTICUT CHILDRENS MEDICAL CENTER,HARTFORD,-999999.0,-999999.0
9,725,MANCHESTER MEMORIAL HOSPITAL,MANCHESTER,28.0,196.0


# Joining with city-level positivity data and population data

Further analysis:
- Correlations between these? Is there a relationship between % positivity and hospital capacity?

In [34]:
df_pop_pos = pd.read_csv('town_rate (5).csv')

In [42]:
df_pop_pos['town'] = df_pop_pos['town'].str.upper()

In [45]:
df_covid_by_hosp.merge(df_pop_pos, left_on='city', right_on='town')

,hospital_name,city,total_adult_patients_hospitalized_confirmed_and_suspected_covid_7_day_avg,total_adult_patients_hospitalized_confirmed_and_suspected_covid_7_day_sum,Unnamed: 0,town_no,town,pop,casesweek1,casesweek2,totalcases,caserate,ratecategory,totaltests,percentpositive,reportperiodstartdate,reportperiodenddate,updatedate
0,DANBURY HOSPITAL,DANBURY,74.7,523.0,33,34,DANBURY,84730,513,706,1219,102.8,"4. 15 or more cases per 100,000",10093,14.6,2020-11-22T00:00:00.000,2020-12-05T00:00:00.000,2020-12-10
1,NORWALK HOSPITAL,NORWALK,47.0,329.0,102,103,NORWALK,89047,326,434,760,61.0,"4. 15 or more cases per 100,000",9997,9.5,2020-11-22T00:00:00.000,2020-12-05T00:00:00.000,2020-12-10
2,ST VINCENT'S MEDICAL CENTER,BRIDGEPORT,56.4,395.0,14,15,BRIDGEPORT,144900,728,805,1533,75.6,"4. 15 or more cases per 100,000",15340,12.3,2020-11-22T00:00:00.000,2020-12-05T00:00:00.000,2020-12-10
3,BRIDGEPORT HOSPITAL,BRIDGEPORT,138.7,971.0,14,15,BRIDGEPORT,144900,728,805,1533,75.6,"4. 15 or more cases per 100,000",15340,12.3,2020-11-22T00:00:00.000,2020-12-05T00:00:00.000,2020-12-10
4,GREENWICH HOSPITAL ASSOCIATION -,GREENWICH,33.4,234.0,56,57,GREENWICH,62727,135,154,289,32.9,"4. 15 or more cases per 100,000",6344,5.2,2020-11-22T00:00:00.000,2020-12-05T00:00:00.000,2020-12-10
5,STAMFORD HOSPITAL,STAMFORD,44.3,310.0,134,135,STAMFORD,129775,543,685,1228,67.6,"4. 15 or more cases per 100,000",14144,10.6,2020-11-22T00:00:00.000,2020-12-05T00:00:00.000,2020-12-10
6,HOSPITAL FOR SPECIAL CARE,NEW BRITAIN,-999999.0,10.0,88,89,NEW BRITAIN,72453,396,545,941,92.8,"4. 15 or more cases per 100,000",9308,12.2,2020-11-22T00:00:00.000,2020-12-05T00:00:00.000,2020-12-10
7,"HOSPITAL OF CENTRAL CONNECTICUT, THE",NEW BRITAIN,48.6,340.0,88,89,NEW BRITAIN,72453,396,545,941,92.8,"4. 15 or more cases per 100,000",9308,12.2,2020-11-22T00:00:00.000,2020-12-05T00:00:00.000,2020-12-10
8,HARTFORD HOSPITAL,HARTFORD,121.6,851.0,63,64,HARTFORD,122587,589,838,1427,83.1,"4. 15 or more cases per 100,000",16142,10.7,2020-11-22T00:00:00.000,2020-12-05T00:00:00.000,2020-12-10
9,CONNECTICUT CHILDRENS MEDICAL CENTER,HARTFORD,-999999.0,-999999.0,63,64,HARTFORD,122587,589,838,1427,83.1,"4. 15 or more cases per 100,000",16142,10.7,2020-11-22T00:00:00.000,2020-12-05T00:00:00.000,2020-12-10
